In [1]:
import pandas as pand
from arcgis.gis import GIS
from arcgis.features import Table,GeoAccessor, GeoSeriesAccessor
from IPython.display import display
gis = GIS()
#Medidas Sanitarias:0b944d9bf1954c71a7fae96bdddee464
Medidas=gis.content.get('0b944d9bf1954c71a7fae96bdddee464')
Capas=Medidas.layers
Cuarentenas=Capas[1]
Medidas

<Item title:"COVID-19 Chile: Medidas Sanitarias" type:Feature Layer Collection owner:lzamudio@soporta.cl_soportachile>

In [2]:
#Ejemplo trabajando con toda la capa como Spatial Enabled Data Frame
sdf = pand.DataFrame.spatial.from_layer(Cuarentenas)
sdf.head()

,OBJECTID,CuarentenaID,Nombre,Estado,Alcance,FInicio,FTermino,Cut_Com,Detalle,GlobalID,created_user,created_date,last_edited_user,last_edited_date,Shape__Area,Shape__Length,SHAPE
0,1,1,Arica,1,2,2020-04-17 02:00:00,2020-05-08 02:00:00,15101,Corresponde al Radio Urbano de la Ciudad de Ar...,d03bd85c-9e0c-43be-9c53-92b75f0198af,lzamudio@soporta.cl_soportachile,2020-04-23 03:43:38.051,lzamudio@soporta.cl_soportachile,2020-04-28 16:09:59.348,2.805042e+07,48722.300013,"{""rings"": [[[-7828315.653, -2093716.3584], [-7..."
1,2,2,Santiago Norte,1,4,2020-04-13 09:00:00,2020-05-08 02:00:00,13101,Corresponde a la extensión de la cuarentena en...,1d643d6d-8955-4188-ae14-6fb4a3f1addc,lzamudio@soporta.cl_soportachile,2020-04-23 03:43:38.051,lzamudio@soporta.cl_soportachile,2020-04-28 16:10:04.974,1.993922e+07,21599.351343,"{""rings"": [[[-7868118.8275, -3952887.3704], [-..."
2,3,3,Puente Alto Poniente,1,4,2020-04-10 02:00:00,2020-05-08 02:00:00,13201,Corresponde al sector delimitado por la Av. Co...,52d8b31a-e552-4d86-a9bd-14c39bd647c4,lzamudio@soporta.cl_soportachile,2020-04-23 03:43:38.051,lzamudio@soporta.cl_soportachile,2020-04-28 16:10:11.080,4.236427e+07,30428.051523,"{""rings"": [[[-7858312.0256, -3971194.6136], [-..."
3,4,4,El Bosque,1,1,2020-04-17 02:00:00,2020-05-08 02:00:00,13105,None,a8e2e38d-6fe2-4cc1-a77c-e8f98afd4f17,lzamudio@soporta.cl_soportachile,2020-04-23 03:43:38.051,lzamudio@soporta.cl_soportachile,2020-04-28 16:10:24.118,2.062271e+07,18788.870751,"{""rings"": [[[-7868656.278, -3967081.1805], [-7..."
4,5,5,San Bernardo Nororiente,1,4,2020-04-17 02:00:00,2020-05-08 02:00:00,13401,Corresponde al sector delimitado por los limit...,b82acfd9-366c-4201-a777-0ad089db1fa4,lzamudio@soporta.cl_soportachile,2020-04-23 03:43:38.051,lzamudio@soporta.cl_soportachile,2020-04-28 16:10:28.337,1.632622e+07,24929.145720,"{""rings"": [[[-7866599.0938, -3974762.2281], [-..."


In [5]:
CuarTotal=Cuarentenas.query(out_fields='CuarentenaID,Nombre,Estado,Alcance,FInicio,FTermino,Cut_Com,Detalle,Shape__Area,Shape__Length',return_geometry=False).sdf
CuarTotal.set_index("OBJECTID", inplace=True)
CuarActivas=CuarTotal[(CuarTotal['Estado'] == 1)|(CuarTotal['Estado'] == 3)]
CuarHistoricas=CuarTotal[(CuarTotal['Estado'] == 2)]
Productos=[CuarTotal,CuarActivas,CuarHistoricas]
for producto in Productos:
    producto.loc[(producto.Estado == 1),'Estado']='Activa'
    producto.loc[(producto.Estado == 2),'Estado']='Histórica'
    producto.loc[(producto.Estado == 3),'Estado']='Futura'
    producto.loc[(producto.Alcance == 1),'Alcance']='Comuna completa'
    producto.loc[(producto.Alcance == 2),'Alcance']='Área Urbana Completa'
    producto.loc[(producto.Alcance == 3),'Alcance']='Área Rural Completa'
    producto.loc[(producto.Alcance == 4),'Alcance']='Sector Específico'
    producto.rename(columns={'CuarentenaID': 'ID', 'FInicio': 'Fecha de Inicio','FTermino': 'Fecha de Término','Cut_Com': 'Código CUT Comuna','Shape__Area': 'Superficie en m2','Shape__Length': 'Perímetro en m'},inplace=True)

,ID,Nombre,Estado,Alcance,Fecha de Inicio,Fecha de Término,Código CUT Comuna,Detalle,Superficie en m2,Perímetro en m
OBJECTID,,,,,,,,,,
6,6,Chillán,Histórica,Área Urbana Completa,2020-03-31 01:00:00,2020-04-24 02:00:00,16101,None,3.783543e+07,49337.02457


In [8]:
CuarTotal.to_csv(path_or_buf="../output/Cuarentenas-Totales.csv",index=False)
CuarActivas.to_csv(path_or_buf="../output/Cuarentenas-Activas.csv",index=False)
CuarHistoricas.to_csv(path_or_buf="../output/Cuarentenas-Historicas.csv",index=False)